# GW170817 - Visualização dos dados

### Importações:

In [1]:
import os
import h5py 
import json
import numpy as np
import healpy as hp
import pickle
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
from tqdm import tqdm
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FuncFormatter
from CHIMERA.DataGW import DataLVK
from CHIMERA.Likelihood import LikeLVK
from CHIMERA.astro.mass import pdf_FLAT, dummy_mass
from CHIMERA.astro.rate import phi_PL, dummy_rate
from CHIMERA.cosmo import fLCDM
from CHIMERA.utils import misc, plotting
from astropy.cosmology import FlatLambdaCDM
from scipy.optimize import root
%config InlineBackend.figure_format = "svg"

### Path dos arquivos

In [2]:
# Arquivos originais
dir_LVK       = os.getcwd()
dir_GLADE     = "../../data/GLADE/glade+_GW170817_cutout.hdf5"
dir_GLADE_int = "../../data/GLADE/p_bkg_gauss_smooth_zerr0.001_zres5000_smooth30.pkl"

 ### Seleção dos eventos que serão analisados

In [3]:
data_GW_n = "GW170817"                          
data_GW = DataLVK(dir_LVK).load(data_GW_n)      
data_GW["dL"] /= 1000.                          

GW170817_GWTC-1.hdf5


### Definição dos dados da NGC

In [4]:
ra4993    = 3.44613079                         
dec4993   = -0.40812585                        
z4993     = 0.0091909380476333                 
errz4993  = 0.0002665372033813657              

___

## Visualização e exploração dos arquivos de dados originais

### Abrindo os arquivos
- modo 'r' para os arquivos originais (somente leitura)
- modo 'r+' para a cópia dos arquivos (leitura de modificações)
- modo 'rb' para o .pkl

In [5]:
#ARQUIVOS ORIGINAIS
file_name = "GW170817_GWTC-1.hdf5"                         
file = h5py.File(file_name, 'r') 

file_path = "../../data/GLADE/glade+_GW170817_cutout.hdf5"
file_2 = h5py.File(file_path, 'r')

file_path_pkl = "../../data/GLADE/p_bkg_gauss_smooth_zerr0.001_zres5000_smooth30.pkl"
with open(file_path_pkl, 'rb') as file_pkl:
    dataPKL = pickle.load(file_pkl)

/tmp/ipykernel_7077/2169444026.py:10: DeprecationWarning: Please import `interp1d` from the `scipy.interpolate` namespace; the `scipy.interpolate.interpolate` namespace is deprecated and will be removed in SciPy 2.0.0.
  dataPKL = pickle.load(file_pkl)


### "GW170817_GWTC-1.hdf5" - DADOS DO EVENTO GW170817

In [7]:
# List all group names, dataset names, and attributes in the file
def list_hdf5_items(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"Group: {name}")
        for key, value in obj.attrs.items():
            print(f"  Attribute: {key} = {value}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name}")
        for key, value in obj.attrs.items():
            print(f"  Attribute: {key} = {value}")
        data = obj[:]  # Read data
        #print(f"  Data shape: {data.shape}")  # Print shape of data
        #print(f"  Data:")
        #print(data)  # Print the data itself

file.visititems(list_hdf5_items)

#........................................................
dataset = file['Overall_posterior']
data = dataset[:]

#print("Campos do registro:", data.dtype.names)

Dataset: Overall_posterior


#### Dados da GW que serão úteis no restante da análise

In [8]:
luminosity_distance = data['luminosity_distance_Mpc']
right_ascension = data['right_ascension']
declination = data['declination']
m1 = data['m1_detector_frame_Msun']
m2 = data['m2_detector_frame_Msun']

In [ ]:
# Histograma para a distância de luminosidade
plt.figure(figsize=(6, 3))
plt.hist(luminosity_distance, bins=50, alpha=0.75)
plt.title('Luminosity Distance distribution')
plt.xlabel('Luminosity Distance (Mpc)')
plt.ylabel('Frequência')
plt.grid(True)
#plt.show()

#Histograma combinado das massas m1 e m2
plt.figure(figsize=(6, 3))
plt.hist(m1, bins=50, alpha=0.5, label='m1_detector_frame_Msun', color='blue')
plt.hist(m2, bins=50, alpha=0.5, label='m2_detector_frame_Msun', color='red')
plt.title('Distribuições de m1 e m2 no referencial do detector (Msun)')
plt.xlabel('Massa (Msun)')
plt.ylabel('Frequência')
plt.legend()
plt.grid(True)
#plt.show()

#### 3D plot: dL, ra, dec

In [9]:
fig = plt.figure(figsize=(14, 9))
ax = fig.add_subplot(111, projection='3d')


sc = ax.scatter(luminosity_distance,right_ascension,declination, c=luminosity_distance, 
                cmap='plasma', marker='o')

plt.title('Luminosity distance, right ascension and declination for GW170817')
ax.set_xlabel('Luminosity Distance (Mpc)')
ax.set_ylabel('Right Ascension (rad)')
ax.set_zlabel('Declination (rad)')
plt.tight_layout()
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)
#plt.show()
plt.savefig('plot1.jpg')
plt.close()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=luminosity_distance,
    y=right_ascension,
    z=declination,
    mode='markers',
    marker=dict(
        size=5,
        color=luminosity_distance,  # Definindo a cor dos pontos com base na distância de luminosidade
        colorscale='Viridis',       
        opacity=0.8
    )
))

# Atualizando os layouts dos eixos
fig.update_layout(
    scene=dict(
        xaxis_title='Luminosity Distance (Mpc)',
        yaxis_title='Right Ascension (rad)',
        zaxis_title='Declination (rad)'
    ),
    title='3D Scatter Plot of Luminosity Distance, Right Ascension, and Declination'
)

fig.show()

___

### "../../data/GLADE/glade+_GW170817_cutout.hdf5" - DADOS DO CATÁLOGO GLADE+ COM CORTE

In [ ]:
file_path = "../../data/GLADE/glade+_GW170817_cutout.hdf5"
file = h5py.File(file_path, 'r')

# List all group names, dataset names, and attributes in the file
def list_hdf5_items_2(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"Group: {name}")
        for key, value in obj.attrs.items():
            print(f"  Attribute: {key} = {value}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name}")
        for key, value in obj.attrs.items():
            print(f"  Attribute: {key} = {value}")
        dataCAT = obj[:]  # Read data
        #print(f"  Data shape: {dataCAT.shape}")  # Print shape of data
        #print(f"  Data:")
        #print(dataCAT)  # Print the data itself

file.visititems(list_hdf5_items_2)

#### Dados do catálogo que serão úteis no restante da análise

In [11]:
z_data = file_2['z'][:]
sigmaz_data = file_2['sigmaz'][:]
dec_data = file_2['dec'][:]
ra_data = file_2['ra'][:]

In [ ]:
print("O valor mínimo de z é:", min(z_data))
print("O valor máximo de z é:",max(z_data))

print("O valor mínimo de sigma_z é:",min(sigmaz_data))
print("O valor máximo de sigma_z é:",max(sigmaz_data))

print("O valor mínimo de dec é:",min(dec_data))
print("O valor máximo de dec é:",max(dec_data))

print("O valor mínimo de ra é:",min(ra_data))
print("O valor máximo de ra é:",max(ra_data))

In [ ]:
#Histograma dos dados de 'z'
plt.figure(figsize=(6, 3))
plt.hist(z_data, bins=50, alpha=0.75, label='z', color='blue')
plt.xlabel('z values')
plt.ylabel('Frequency')
plt.title('Histogram of z Dataset')
plt.legend()
plt.grid(True)
plt.show()

#Histograma dos dados de 'sigmaz'
plt.figure(figsize=(6, 3))
plt.hist(sigmaz_data, bins=50, alpha=0.75, label='sigmaz', color='orange')
plt.xlabel('sigmaz values')
plt.ylabel('Frequency')
plt.title('Histogram of sigmaz Dataset')
plt.legend()
plt.grid(True)
plt.show()

#### KDE de z

In [ ]:
from scipy.stats import gaussian_kde

# Função para plotar KDE
def plot_kde(data, label, color):
    kde = gaussian_kde(data)
    x_vals = np.linspace(min(data), max(data), 1000)
    y_vals = kde(x_vals)
    plt.plot(x_vals, y_vals, label=label, color=color)

#Curva de densidade de probabilidade de 'z'
plt.figure(figsize=(6, 3))
plot_kde(z_data, 'z', 'blue')
plt.xlabel('z values')
plt.ylabel('Probability Density')
plt.title('Probability Density Function of z Dataset')
plt.legend()
plt.grid(True)
plt.show()

#### 3D plot: z, ra, dec

In [17]:
fig = plt.figure(figsize=(14, 9))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(z_data,ra_data, dec_data, c=z_data, cmap='viridis', marker='o')
ax.set_xlabel('Redshift (z)')
ax.set_ylabel('Right Ascension (ra)')
ax.set_zlabel('Declination (dec)')
ax.set_title('Redshift, right ascension and declination data from GLADE+ catalog')
plt.tight_layout()
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)
#plt.show()
plt.savefig('plot2.jpg')
plt.close()

In [ ]:
scatter = go.Scatter3d(
    x=z_data,       
    y=ra_data,  
    z=dec_data,     
    mode='markers',
    marker=dict(
        size=5,
        color=z_data,  # Cor baseado no redshift
        colorscale='Viridis',
        opacity=0.8
    )
)

layout = go.Layout(
    title='Gráfico 3D de Declinação, Ascensão Reta e Redshift',
    scene=dict(
        xaxis=dict(title='Redshift (z)'),
        yaxis=dict(title='Ascensão Reta (ra)'),
        zaxis=dict(title='Declinação (dec)')
    )
)

fig = go.Figure(data=[scatter], layout=layout)
pio.show(fig)

### "p_bkg_gauss_smooth_zerr0.001_zres5000_smooth30.pkl" - DADOS DO GLADE+ (COMPLETO) SUAVIZADO

In [19]:
# Visualizar o conteúdo
print("Tipo de dado:", type(dataPKL))

if isinstance(dataPKL, dict):
    for key, value in dataPKL.items():
        print(f"\nChave: {key}")
        print("Valor:", value)
elif isinstance(dataPKL, list):
    for i, item in enumerate(dataPKL):
        print(f"\nItem {i}:")
        print(item)
else:
    print(dataPKL)

Tipo de dado: <class 'scipy.interpolate._interpolate.interp1d'>


In [ ]:
# Verificar o tipo de dado e propriedades
print("Tipo de dado:", type(dataPKL))

# Se for um objeto interp1d, podemos inspecionar mais
if isinstance(dataPKL, interp1d):
    print("Informações sobre a interpolação:")
    print("Interpolante:", dataPKL)
    print("Intervalo de definição:", dataPKL.x[0], "a", dataPKL.x[-1])
    print("Intervalo de valores:", dataPKL.y[0], "a", dataPKL.y[-1])
else:
    print("O objeto carregado não é uma função de interpolação interp1d")

In [ ]:
if isinstance(dataPKL, interp1d):
    # Definir o intervalo de redshift (ajuste conforme necessário)
    z_min = 0  # Redshift mínimo
    z_max = 1  # Redshift máximo
    
    # Gerar pontos no intervalo de redshift
    z_points = np.linspace(z_min, z_max, 5000)
    
    # Avaliar a função interpolante nesses pontos
    interpolated_values = dataPKL(z_points)
    
    # Plotar a interpolação
    plt.figure(figsize=(6, 3))
    plt.plot(z_points, interpolated_values, '-',label='Interpolação Suavizada')
    plt.xlabel('Redshift (z)')
    plt.ylabel('Valor Interpolado')
    plt.title('Interpolação Suavizada do Catálogo GLADE+')
    plt.legend()
    plt.show()
else:
    print("O objeto carregado não é uma função de interpolação interp1d")

In [ ]:
# Plotar a interpolação apenas até z= 0,01
plt.figure(figsize=(6, 3))
plt.plot(z_points, interpolated_values, '-', label='Interpolação Suavizada')
plt.xlabel('Redshift (z)')
plt.ylabel('Number Density of Galaxies (?)')
plt.title('Interpolação Suavizada do Catálogo GLADE+')
plt.xlim(0,0.1)
plt.ylim(0,0.1)
plt.legend()
plt.show()

In [ ]:
densidade_cortada = file_2.get('densidade_de_numero', np.ones_like(z_data))  # Supondo que os dados cortados contenham essa informação

# Gerar pontos no intervalo de redshift da interpolação completa
z_points = np.linspace(dataPKL.x[0], dataPKL.x[-1], 5000)

# Avaliar a função interpolante nesses pontos
interpolated_values = dataPKL(z_points)

# Plotar os dados originais cortados
plt.figure(figsize=(8, 4))
plt.scatter(z_data, densidade_cortada, color='blue', label='Dados Originais Cortados')

# Plotar a interpolação suavizada
plt.plot(z_points, interpolated_values, color='red', label='Interpolação Suavizada')

# Configurar o gráfico
plt.xlabel('Redshift (z)')
plt.ylabel('Number Density of Galaxies (?)')
#plt.title('Comparação entre Dados Originais Cortados e Interpolação Suavizada')
plt.xlim(0,0.2)
plt.legend()
plt.show()

In [ ]:
densidade_cortada = file_2.get('densidade_de_numero', np.ones_like(z_data))

# Plotar os dados originais cortados
plt.figure(figsize=(8, 4))
plt.scatter(z_data, densidade_cortada, color='blue', label='Dados Originais Cortados')

# Configurar o gráfico
plt.xlabel('Redshift (z)')
plt.ylabel('Densidade de Número de Galáxias')
plt.title('Distribuição de Galáxias nos Dados Cortados')
plt.legend()
plt.show()

### "GWTC-confident.json" - TODAS AS DETECÇÕES

In [12]:
# Definir o caminho para o arquivo .json
dir_LVK = os.getcwd()  # ou o caminho onde o arquivo está localizado
json_file_path = os.path.join(dir_LVK, "GWTC-confident.json")

# Carregar o conteúdo do arquivo .json
with open(json_file_path, 'r') as f:
    json_data = json.load(f)

# Exibir o conteúdo do JSON
#print(json.dumps(json_data, indent=4))

O arquivo .json é usado para fornecer metadados e ajudar a localizar os arquivos .hdf5 corretos para cada evento. Ele contém informações que permitem a classe DataLVK identificar e carregar os arquivos de dados apropriados. 

### Fechando os arquivos 

In [13]:
#Arquivos originais
file.close()
file_2.close()
file_pkl.close()

___

___

### Combinando dados da GW com o catálogo 

#### Convertendo dL em z - tomando LCDM:

In [27]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from scipy.optimize import root

# Definir o modelo cosmológico 
H0 = 70      #km/s/Mpc
Om0 = 0.25  
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

# Função para calcular a diferença entre a distância de luminosidade fornecida e a calculada para um dado redshift
def redshift_from_luminosity_distance(d_l, z_initial_guess=0.02):
    def f(z):
        return cosmo.luminosity_distance(z).value - d_l
    sol = root(f, z_initial_guess)
    if sol.success:
        return sol.x[0]
    else:
        raise ValueError("A solução não foi encontrada")


# Função para converter uma lista de distâncias de luminosidade em redshift
def convert_distances_to_redshifts(distances):
    redshifts = []
    for d_l in distances:
        try:
            z = redshift_from_luminosity_distance(d_l)
            redshifts.append(z)
        except ValueError as e:
            print(f"Erro ao calcular o redshift para a distância {d_l} Mpc: {e}")
            redshifts.append(np.nan)  # Adiciona NaN em caso de erro
    return redshifts


redshifts_GW = convert_distances_to_redshifts(luminosity_distance)

#### Visualização 3D - z, ra, dec

In [ ]:
fig = plt.figure(figsize=(14, 9))
ax = fig.add_subplot(111, projection='3d')

# Plotar os dados do primeiro conjunto
sc1 = ax.scatter(redshifts_GW, right_ascension, declination, c=luminosity_distance, 
                 cmap='plasma', marker='o', label='GW170817')

# Plotar os dados do segundo conjunto
sc2 = ax.scatter(z_data, ra_data, dec_data, c=z_data, cmap='viridis', marker='^', label='GLADE+')

# Adicionar labels aos eixos
ax.set_xlabel('Redshift (z)')
ax.set_ylabel('Right ascension (ra)')
ax.set_zlabel('Declination (dec)')
ax.set_title('Combined data from GLADE+ and GW170817')
plt.tight_layout()
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)
# Adicionar legenda
ax.legend()

# Mostrar o gráfico
plt.show()
#plt.savefig('plot3.jpg')
#plt.close()

In [ ]:
# Realizar a amostragem dos dados do catálogo
sample_indices = np.random.choice(len(z_data), size=len(z_data)//50, replace=False)
z_data_sampled = z_data[sample_indices]
sigmaz_data_sampled = sigmaz_data[sample_indices]
dec_data_sampled = dec_data[sample_indices]
ra_data_sampled = ra_data[sample_indices]

# Plotar os dados
fig = plt.figure(figsize=(14, 9))
ax = fig.add_subplot(111, projection='3d')

# Plotar os dados do primeiro conjunto
sc1 = ax.scatter(redshifts_GW, right_ascension, declination, c=luminosity_distance, 
                 cmap='plasma', marker='o', label='GW170817')

# Plotar os dados do segundo conjunto
sc2 = ax.scatter(z_data_sampled, ra_data_sampled, dec_data_sampled, c=z_data_sampled, 
                 cmap='viridis', marker='^', label='GLADE+ (Amostragem)')

# Adicionar labels aos eixos
ax.set_xlabel('Redshift (z)')
ax.set_ylabel('Right ascension (ra)')
ax.set_zlabel('Declination (dec)')
ax.set_title('Combined data from GLADE+ and GW170817 (Sampled)')
plt.tight_layout()
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)
# Adicionar legenda
ax.legend()

# Salvar o gráfico
#plt.savefig('plot4.jpg')
#plt.close()
plt.show()

In [ ]:
# Criar o gráfico interativo
fig = go.Figure()

# Adicionar os dados da GW170817
fig.add_trace(go.Scatter3d(
    x=redshifts_GW,
    y=right_ascension,
    z=declination,
    mode='markers',
    marker=dict(
        size=5,
        color=luminosity_distance,
        colorscale='Plasma',
        opacity=0.8
    ),
    name='GW170817'
))

# Adicionar os dados amostrados do catálogo GLADE+
fig.add_trace(go.Scatter3d(
    x=z_data_sampled,
    y=ra_data_sampled,
    z=dec_data_sampled,
    mode='markers',
    marker=dict(
        size=5,
        color=z_data_sampled,
        colorscale='Viridis',
        opacity=0.8
    ),
    name='GLADE+ (Amostragem)'
))

# Atualizar os layouts do gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='Redshift (z)',
        yaxis_title='Right Ascension (ra)',
        zaxis_title='Declination (dec)'
    ),
    title='Combined data from GLADE+ and GW170817 (Sampled)',
    legend_title_text='Datasets',
    width=1400,  # Largura da célula
    height=800   # Altura da célula
)

# Mostrar o gráfico interativo
fig.show()

___

### Testes com as alterações na declinação - ver o outro notebook

### Análise dos parâmetros cosmológicos - ver o outro notebook